In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader, TextLoader
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [4]:
# 아래 코드는 옵션이다.
# vecstore 를 신규로 만드는 부분.
# vecstore가 이미 만들어져 있다면 아래에서, 로컬에서 가져오는 코드를 실행하라.

# Load, chunk and index the contents of the blog.
loader = TextLoader('../data/law_qa_dataset.csv', encoding='utf-8')
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100, add_start_index=True)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(), persist_directory="../data/chroma_db")


In [5]:
# 해당 부분은 이미 임베딩된 데이터를 로드하는 부분이다.

vectorstore = Chroma(persist_directory='../data/chroma_db', embedding_function=OpenAIEmbeddings())

In [5]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [6]:
retrieved_docs = retriever.invoke("실업자도 지역별 노동조합에 가입할 수 있나요?")

In [7]:
len(retrieved_docs)

6

In [8]:
print(retrieved_docs[0].page_content)

"저는 현재 직장을 구하는 중에 있는 실업자로서, 지역별 노동조합이 생긴다고 하기에 이에 가입하고 싶습니다. 가입이 가능한지요?",,"직장생활을 하는 근로자를 보호하기 위한 법률로는 「근로기준법」이 있고, 근로자의 단결권·단체교섭권·단체행동권이라는 근로3권을 보장하기 위한 법률로는 「노동조합 및 노동관계조정법」(이하 ‘노조법’이라 한다)이 있습니다. 「근로기준법」제2조 제1항 제1호에 의하면 ‘근로자’란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 자를 말하며, 「노조법」제2조 제1호에 의하면 ‘근로자’라 함은 직업의 종류를 불문하고 임금·급료 기타 이에 준하는 수입에 의하여 생활하는 자로 규정하고 있습니다. 또한 「노조법」제2조 제4호는 “‘노동조합’은 근로자가 주체가 되어 자주적으로 단결하여 근로조건의 유지·개선 기타 근로자의 경제적·사회적 지위의 향상을 도모함을 목적으로 조직하는 단체 또는 그 연합단체를 말하며 다만, 같은 호 라목에 의하면 근로자가 아닌 자의 가입을 허용하는 경우 노동조합으로 보지 않는다.”라고 규정하고 있습니다.이와 관련하여 지역별 노동조합이 그 구성원으로 ‘구직중인 여성 노동자’를 포함하여 노동조합설립신고를 한 것에 대하여 판례는 “근로기준법은 ‘현실적으로 근로를 제공하는 자에 대하여 국가의 관리·감독에 의한 직접적인 보호의 필요성이 있는가’라는 관점에서 개별적 노사관계를 규율할 목적으로 제정된 것인 반면에, 노조법은 ‘노무공급자들 사이의 단결권 등을 보장해 줄 필요성이 있는가’라는 관점에서 집단적 노사관계를 규율할 목적으로 제정된 것으로 그 입법목적에 따라 근로자의 개념을 상이하게 정의하고 있는 점, 일정한 사용자에의 종속관계를 조합원의 자격요건으로 하는 기업별 노동조합의 경우와는 달리 산업별·직종별·지역별 노동조합 등의 경우에는 원래부터 일정한 사용자에의 종속관계를 조합원의 자격요건으로 하는 것이 아닌 점에 비추어, 노조법 제2조 제1호 및 제4호 (라)목 본문에서 말하는 ‘근로자’에는 특정한


# 새로 질문이 들어왔을 때

In [9]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [10]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [11]:
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()
example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [12]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

'예, 지역별 노동조합에는 구직중인 실업자도 포함될 수 있습니다. 노동조합법은 일시적으로 실업 상태에 있는 자나 구직 중인 자도 노동조합에 가입할 수 있음을 명시하고 있습니다. 이에 따라 실업자도 지역별 노동조합에 가입할 수 있습니다.'

In [14]:
#rag_chain.invoke("실업자도 지역별 노동조합에 가입할 수 있나요?")

for chunk in rag_chain.stream("실업자도 지역별 노동조합에 가입할 수 있나요?"):
    print(chunk, end="", flush=True)

노동조합에 가입 가능한 범위에는 일시적으로 실업 상태에 있는 자나 구직 중인 자도 포함될 수 있습니다. 따라서 실업자도 지역별 노동조합에 가입할 수 있을 것으로 보입니다. 판례에 따르면 노동조합에 구직 중인 실업자가 포함될 수 있다는 내용이 있습니다.

In [17]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)
chain.invoke("실업자도 지역별 노동조합에 가입할 수 있나요?")

ChatPromptValue(messages=[HumanMessage(content='You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don\'t know the answer, just say that you don\'t know. Use three sentences maximum and keep the answer concise.\nQuestion: 실업자도 지역별 노동조합에 가입할 수 있나요? \nContext: "저는 현재 직장을 구하는 중에 있는 실업자로서, 지역별 노동조합이 생긴다고 하기에 이에 가입하고 싶습니다. 가입이 가능한지요?",,"직장생활을 하는 근로자를 보호하기 위한 법률로는 「근로기준법」이 있고, 근로자의 단결권·단체교섭권·단체행동권이라는 근로3권을 보장하기 위한 법률로는 「노동조합 및 노동관계조정법」(이하 ‘노조법’이라 한다)이 있습니다. 「근로기준법」제2조 제1항 제1호에 의하면 ‘근로자’란 직업의 종류와 관계없이 임금을 목적으로 사업이나 사업장에 근로를 제공하는 자를 말하며, 「노조법」제2조 제1호에 의하면 ‘근로자’라 함은 직업의 종류를 불문하고 임금·급료 기타 이에 준하는 수입에 의하여 생활하는 자로 규정하고 있습니다. 또한 「노조법」제2조 제4호는 “‘노동조합’은 근로자가 주체가 되어 자주적으로 단결하여 근로조건의 유지·개선 기타 근로자의 경제적·사회적 지위의 향상을 도모함을 목적으로 조직하는 단체 또는 그 연합단체를 말하며 다만, 같은 호 라목에 의하면 근로자가 아닌 자의 가입을 허용하는 경우 노동조합으로 보지 않는다.”라고 규정하고 있습니다.이와 관련하여 지역별 노동조합이 그 구성원으로 ‘구직중인 여성 노동자’를 포함하여 노동조합설립신고를 한 것에 대하여 판례는 “근로기준법은 ‘현실적으로 근로를 제공하는 자에 대하여 국가의 관리·감독에 의한 직접적

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            For questions related to the law, print it as 'law', for everything else, print it as 'etc'.
            e.g. worker rights -> law
            e.g. entertainment industry articles, sports related news -> etc
            """
        ),
        ("user", "{input}")
    ]
)

chain = (
    prompt
    | llm
    | StrOutputParser()
)
chain.invoke("실업자도 지역별 노동조합에 가입할 수 있나요?")

'law'

In [15]:
chain.invoke("뉴진스 사태에 대해서 알려줘")

'etc'